In [3]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import pickle

In [4]:
#importing data set 
df = pd.read_csv("diabetes.csv")

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
#for indepth data analysis
profile = ProfileReport(df)
profile.to_file("Report.html")

In [7]:
##Replacing zeros from columns like Glucose, BloodPressure, SkinThickness, Insulin, BMI
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [8]:
#making copy of dataframe 
dfc = df

In [9]:
#replacing NaN values by mean
dfc.fillna(dfc.mean(),inplace=True)

In [10]:
dfc.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.00000,155.548223,33.6,0.627,50,1
1,1,85.0,66.0,29.00000,155.548223,26.6,0.351,31,0
2,8,183.0,64.0,29.15342,155.548223,23.3,0.672,32,1
3,1,89.0,66.0,23.00000,94.000000,28.1,0.167,21,0
4,0,137.0,40.0,35.00000,168.000000,43.1,2.288,33,1


In [11]:
#generating training and validation data
from sklearn.model_selection import train_test_split
X = dfc.iloc[:,:8]
y = dfc.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=50)
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('Accuracy metrics', metrics.accuracy_score(y_test, y_pred))
print('Confusion metrics', metrics.confusion_matrix(y_test, y_pred))

Mean Absolute Error: 0.21428571428571427
Mean Squared Error: 0.21428571428571427
Root Mean Squared Error: 0.4629100498862757
Accuracy metrics 0.7857142857142857
Confusion metrics [[83 16]
 [17 38]]


In [13]:
from sklearn import tree
DT = tree.DecisionTreeClassifier()
DT.fit(X_train, y_train)
y_pred = DT.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('Accuracy metrics', metrics.accuracy_score(y_test, y_pred))
print('Confusion metrics', metrics.confusion_matrix(y_test, y_pred))

Mean Absolute Error: 0.3051948051948052
Mean Squared Error: 0.3051948051948052
Root Mean Squared Error: 0.5524443910429403
Accuracy metrics 0.6948051948051948
Confusion metrics [[73 26]
 [21 34]]


In [14]:
#rescaling data for logistic regression
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
rescaled_X_train = scaler.fit_transform(X_train)
rescale_X_test = scaler.fit_transform(X_test)


In [15]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(rescaled_X_train, y_train)
y_pred = LR.predict(rescale_X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('Accuracy metrics', metrics.accuracy_score(y_test, y_pred))
print('Confusion metrics', metrics.confusion_matrix(y_test, y_pred))

Mean Absolute Error: 0.2792207792207792
Mean Squared Error: 0.2792207792207792
Root Mean Squared Error: 0.5284134548067254
Accuracy metrics 0.7207792207792207
Confusion metrics [[75 24]
 [19 36]]


In [16]:
#out of all we are getting higher accurary for Random Forest Classifier
pickle.dump(RF, open('rfc_diabetes.pkl', 'wb'))


In [18]:
##to determine feature importance
importances = RF.feature_importances_
std = np.std([tree.feature_importances_ for tree in RF.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

Feature ranking:
1. feature 1 (0.247549)
2. feature 5 (0.168809)
3. feature 7 (0.148805)
4. feature 6 (0.115488)
5. feature 4 (0.091384)
6. feature 2 (0.081044)
7. feature 3 (0.075560)
8. feature 0 (0.071360)
